In [34]:
import pandas as pd

## Data: How is the .csv organized?

For each .csv file the following apply:
* each row represents the data of a single vehicle
* the first 10 columns in the 1st row include the columns’ names
* the first 4 columns include information about the trajectory like the unique trackID, the type of vehicle, the distance traveled in meters and the average speed of the vehicle in km/h
* the last 6 columns are then repeated every 6 columns based on the time frequency. For example, column_5 contains the latitude of the vehicle at time column_10, and column­­­_11 contains the latitude of the vehicle at time column_16.
* Speed is in km/h, Longitudinal and Lateral Acceleration in m/sec2 and time in seconds.

In [35]:
# 1. lets blindly try to read the data into a dataframe

# df = pd.read_csv("../data/test_data.csv", sep=";")

The error is caused by the fact that rows do **not** have equal number of columns. 

In [36]:
with open("../data/test_data.csv", 'r') as file:
    lines = file.readlines()

In [37]:
print(f"The number of rows/lines is {len(lines)}")

The number of rows/lines is 853


In [38]:
print(lines[0]) # column names
print(lines[0].strip('\n').strip().strip(';').split(';')) # columns names as a list

track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time

['track_id', ' type', ' traveled_d', ' avg_speed', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [39]:
lines_as_lists = [line.strip('\n').strip().strip(';').split(';') for line in lines]
len(lines_as_lists)

853

In [40]:
print(f"the number of fields in row 1 is {len(lines_as_lists[1])}, row 2 is {len(lines_as_lists[2])}")

the number of fields in row 1 is 3010, row 2 is 10510


The number of fields in each row is = 4 + 6 * n

* The first 4 unique values are: track_id; type; traveled_d; avg_speed
* The remaining fields are repeated sequences of: 
  * lat_1; lon_1; speed_1; lon_acc_1; lat_acc_1; time_1
  * lat_2; lon_2; speed_2; lon_acc_2; lat_acc_2; time_2
  * ...
  * lat_n; lon_n; speed_n; lon_acc_n; lat_acc_n; time_n

* n depends on the time frequency. It is different for each row.


In [41]:
no_field_max = 0

for row in lines_as_lists:
    if len(row) > no_field_max:
        no_field_max = len(row)

print(f"the maximum number of fields is {no_field_max}")
largest_n = int((no_field_max-4)/6)
print(f"the largest n = {largest_n}")

the maximum number of fields is 98410
the largest n = 16401


We can divide each row into 2 parts:
1.  A list of the first 4 values. 
    - These will correspond to ['track_id', ' type', ' traveled_d', ' avg_speed']
2.  A matrix of size 6*n of the remaining values. 
    - These will correspond to a table with columns [' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time'] and n rows.
    - We will append the 'track_id' value to each row to keep track of the vehicle identity.

In [42]:
cols = lines_as_lists.pop(0)

In [43]:
track_cols = cols[:4]
trajectory_cols = ['track_id'] + cols[4:]

print(track_cols)
print(trajectory_cols)


['track_id', ' type', ' traveled_d', ' avg_speed']
['track_id', ' lat', ' lon', ' speed', ' lon_acc', ' lat_acc', ' time']


In [44]:
track_info = []
trajectory_info = []

for row in lines_as_lists:
    track_id = row[0]

    # add the first 4 values to track_info
    track_info.append(row[:4]) 

    remaining_values = row[4:]
    # reshape the list into a matrix and add track_id
    trajectory_matrix = [ [track_id] + remaining_values[i:i+6] for i in range(0,len(remaining_values),6)]
    # add the matrix rows to trajectory_info
    trajectory_info = trajectory_info + trajectory_matrix


In [45]:
df_track = pd.DataFrame(data= track_info,columns=track_cols)

df_track.head(100)

,track_id,type,traveled_d,avg_speed
0,1,Car,134.88,24.278704
1,2,Car,426.97,21.958339
2,3,Car,206.58,8.263246
3,4,Car,261.45,30.361735
4,5,Taxi,264.12,16.979263
...,...,...,...,...
95,96,Car,86.51,12.976159
96,97,Car,212.20,14.042918
97,98,Car,224.82,11.562238
98,99,Motorcycle,191.57,24.630498


In [46]:
df_trajectory = pd.DataFrame(data= trajectory_info,columns=trajectory_cols)

df_trajectory.head(100)

,track_id,lat,lon,speed,lon_acc,lat_acc,time
0,1,37.979513,23.736025,35.3204,-0.2996,-0.0175,14.000000
1,1,37.979510,23.736027,35.2990,-0.2788,-0.0188,14.040000
2,1,37.979508,23.736030,35.2800,-0.2656,-0.0200,14.080000
3,1,37.979505,23.736032,35.2624,-0.2589,-0.0213,14.120000
4,1,37.979502,23.736035,35.2460,-0.2460,-0.0225,14.160000
...,...,...,...,...,...,...,...
95,1,37.979214,23.736284,39.7003,-0.8160,0.1721,17.800000
96,1,37.979210,23.736287,39.5834,-0.8087,0.1764,17.840000
97,1,37.979207,23.736290,39.4686,-0.7858,0.1806,17.880000
98,1,37.979204,23.736293,39.3582,-0.7474,0.1846,17.920000


### Strip the whitespaces from the column names

In [47]:
df_trajectory.columns = df_trajectory.columns.str.strip()
df_track.columns = df_track.columns.str.strip()

In [48]:
# Define a dictionary with the correct data types for each column
column_types = {
    'track_id': 'int64',
    'lat': 'float64',
    'lon': 'float64',
    'speed': 'float64',
    'lon_acc': 'float64',
    'lat_acc': 'float64',
    'time': 'float64'
}

# Convert the data types of the DataFrame columns
df_trajectory = df_trajectory.astype(column_types)

In [49]:
print(df_trajectory.columns)


Index(['track_id', 'lat', 'lon', 'speed', 'lon_acc', 'lat_acc', 'time'], dtype='object')


In [50]:
df_trajectory.dtypes

track_id      int64
lat         float64
lon         float64
speed       float64
lon_acc     float64
lat_acc     float64
time        float64
dtype: object